In [1]:
# !pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118

In [2]:
# !pip install --upgrade einops nltk pandas higher hydra-core numpy wandb datasets jsonlines matplotlib scikit-learn

In [3]:
# !pip install --upgrade sentencepiece

In [4]:
# !pip install transformers==4.20.1

# Editing a T5 QA model with GRACE

In [5]:
import grace
from grace.editors import GRACE_barebones as GRACE
from grace.utils import tokenize_qa
import torch
import copy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

C:\Users\Jui\miniconda3\envs\grace_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
tokenizer = AutoTokenizer.from_pretrained("google/t5-small-ssm-nq")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 605/605 [00:00<00:00, 603kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 294M/294M [00:11<00:00, 28.0MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [7]:
layer_to_edit = "encoder.block[4].layer[1].DenseReluDense.wo" # Which layer to edit?
init_epsilon = 3.0 # Initial epsilon for GRACE codebook entries
learning_rate = 1.0 # Learning rate with which to learn new GRACE values
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
original_model = copy.deepcopy(model)

In [8]:
# --- wrap model with GRACE ---
edited_model = GRACE(model, layer_to_edit, init_epsilon, learning_rate, device, generation=False)

edit_module T5DenseGatedActDense(
  (wi_0): Linear(in_features=512, out_features=1024, bias=False)
  (wi_1): Linear(in_features=512, out_features=1024, bias=False)
  (wo): Linear(in_features=1024, out_features=512, bias=False)
  (dropout): Dropout(p=0.1, inplace=False)
  (act): NewGELUActivation()
)
layer_name wo
original_layer Linear(in_features=1024, out_features=512, bias=False)
Parameter containing:
tensor([[ 8.4375e-01,  1.3184e-02, -7.4219e-01,  ..., -1.1641e+00,
         -7.5684e-02,  8.5156e-01],
        [ 8.8281e-01, -2.7930e-01, -3.0078e-01,  ...,  7.0312e-01,
          8.8867e-02,  3.0859e-01],
        [ 2.8992e-04,  3.3691e-02,  3.9844e-01,  ..., -2.9688e-01,
          6.9922e-01,  2.5391e-01],
        ...,
        [ 5.3125e-01, -3.2812e-01,  2.5195e-01,  ...,  1.1250e+00,
         -2.2168e-01,  6.7969e-01],
        [ 4.7656e-01,  8.6914e-02, -7.2656e-01,  ...,  5.7422e-01,
         -1.3906e+00,  6.0547e-01],
        [-4.0234e-01, -7.5195e-02,  4.3359e-01,  ...,  5.0781e-02

In [9]:
edited_model.layer

'encoder.block[4].layer[1].DenseReluDense.wo'

In [10]:
# --- Desired edit ---
edit_input = {
    "text": ["How tall is the empire state building?"],
    "labels": ["1,454 feet"],
}

edit_tokens = tokenize_qa(edit_input, tokenizer, device)

# --- Check model's prediction for this edit before applying the edit ---
preds = original_model.generate(edit_tokens["input_ids"]).squeeze()
original_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"Before Editing. Question: {edit_input['text'][0]}. Answer: {original_answer}")

Before Editing. Question: How tall is the empire state building?. Answer: 71 ft


In [14]:
# --- Apply the edit ---
edited_model.edit(edit_tokens)

In [15]:
# --- Check model's prediction for this edit AFTER applying the edit ---
preds = edited_model.generate(edit_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {edit_input['text'][0]}. Answer: {new_answer}")

After Editing. Question: How tall is the empire state building?. Answer: 1,454 feet


In [16]:
# --- Trying slightly different input text ---
test_input = {
    "text": ["how high is the empire state building?"],
    "labels": ["1,454 feet"]
}

test_tokens = tokenize_qa(test_input, tokenizer, device)

preds = edited_model.generate(test_tokens["input_ids"], max_length=20).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {test_input['text'][0]}. Answer: {new_answer}")

After Editing. Question: how high is the empire state building?. Answer: 57 ft


In [17]:
# --- Check if the original and edited model have the same prediction on an unrelated input ---
unrelated_input = {
    "text": ["How tall is the eiffel tower?"],
    "labels": ["1,083 feet"]
}

unrelated_tokens = tokenize_qa(unrelated_input, tokenizer, device)

preds = original_model.generate(unrelated_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"Before Editing. Question: {unrelated_input['text'][0]}. Answer: {new_answer}")

preds = edited_model.generate(unrelated_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {unrelated_input['text'][0]}. Answer: {new_answer}")

Before Editing. Question: How tall is the eiffel tower?. Answer: 157 ft
After Editing. Question: How tall is the eiffel tower?. Answer: 157 ft
